In [ ]:
import numpy as np
import pandas as pd
from datetime import timedelta, datetime, date

#### Feed the downloaded file here

In [ ]:
df = pd.read_csv('transactions.csv')

In [ ]:
df.columns = df.columns.str.strip()
df.head()

In [ ]:
df["Date"] = pd.to_datetime(df["Date"])

In [ ]:
df2 = df

#### Prevent all the liquid fund from showing

In [ ]:
df = df[~df['Name of the Fund'].str.contains("Liquid|Money|Gilt|Cash|Bond|Debt|Short|Gold|Securities|Duration|Income")]
##remove the upper line in case performance of all the mutual fund is required
df = df.loc[~df['Order'].str.contains("sell")]

In [ ]:
today = pd.to_datetime(date.today())

#### Control the number of days to look behind / calculate the Xirr for. for three years we take 365*3 = 1095 days

In [ ]:
df = df.loc[df['Date'] > (pd.to_datetime(date.today() - timedelta(days = 1095)))]

In [ ]:
mf_name = df['Name of the Fund'].unique()

#### See the list of all the mutual funds

In [ ]:
mf_name

In [ ]:
df['Amount (INR)'] = df['Amount (INR)']*-1

In [ ]:
df.head()

#### Create the  function that calculates Xirr ( funciton copied from https://github.com/SunilVeeravalli/XIRR_in_Python/blob/main/xirr.py )

In [ ]:
min_rate = 0
max_rate = 0
def npv(seq_of_rates: np.array, data: pd.DataFrame) -> tuple:
    global min_rate
    global max_rate
    
    for a_rate in seq_of_rates:
        max_date = data['Date'].max()
        data['npv'] = data['Amount'] * ((1 + (a_rate / 100)) ** ((max_date - data['Date']).dt.days / 365))
        
        if data['Amount'].sum() > 0:
            if data['npv'].sum() > 0:
                min_rate = a_rate
            else:
                max_rate = a_rate
                break
        else:
            if data['npv'].sum() < 0:
                min_rate = a_rate
            else:
                max_rate = a_rate
                break
    
    return min_rate, max_rate


def xirr(data: pd.DataFrame) -> float:
    # Finding out which column contains Date and Amount
    try:
        data.iloc[:, 0].astype(float)
        col_names = ['Amount', 'Date']
    except Exception as _:
        col_names = ['Date', 'Amount']
    
    # Renaming the columns accordingly and converting to correct data types
    data.columns = col_names
    data = data.assign(Date = pd.to_datetime(arg = data['Date'], infer_datetime_format = True, dayfirst = True).dt.date,
                       Amount = data['Amount'].astype(float))
    
    if data['Amount'].sum() > 0:
        step_values = [100, 10, 1, 0.01, 0.001, 0.0001]
        stop = 10000
    else:
        step_values = [-100, -10, -1, -0.01, -0.001, -0.0001]
        stop = -10000
    
    start = 0
    for i in range(len(step_values)):
        seq_of_rates = np.arange(start = start, stop = stop, step = step_values[i])
        start, stop = npv(seq_of_rates, data)
    
    return (start + stop) / 2


In [ ]:
solution = []
mf= []
for i in range(0,len(mf_name)):
    mf = df[df['Name of the Fund'] == mf_name[i]]
    mf = mf.append(pd.DataFrame({'Date': today, 'Amount (INR)':mf.Units.sum() * mf['Current Nav'].unique() }), 
                   ignore_index = True, sort = True)
    ready_df = mf[['Date','Amount (INR)']]
    solution.append(xirr(ready_df))

In [ ]:
final_df = pd.DataFrame(list(zip(mf_name.tolist(),solution)))

In [ ]:
final_df.columns = ['Name of the Fund', 'Xirr']

### See the final performance of the mutual fund sorted based on the overall Xirr

In [ ]:
final_df.sort_values('Xirr', ascending = False)